url :
* https://pomber.github.io/covid19/timeseries.json
* https:://covid19api.herokuapp.com/confirmed

In [4]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext


spark = SparkSession \
    .builder \
    .appName("Pysparkexample") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


def changethedate(s):
    tab=s.split("-")
    if len(tab[1])==1:
        tab[1]='0'+tab[1]
    s=tab[0]+'-'+tab[1]+'-'+tab[2]
    return s

def having_covid(str):
    nb_of_cases=int(str)
    if nb_of_cases>0 :
        return 1
    else:
         return 0

import json
import requests
import time
import pandas as pd
from datetime import datetime

url="https://pomber.github.io/covid19/timeseries.json"
            #print(url)
response = requests.get(url)
data_json=response.json()

dd=[]
for d in data_json:
        for l in data_json[d]:
                l['pays']=d
                dd.append(l)

covid_df= spark.createDataFrame(dd)

#changing the format of date
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
func =  udf (lambda x: changethedate(x),StringType())
covid_df=covid_df.withColumn('new_date',func(covid_df['date']))

from pyspark.sql.types import DateType
func =  udf (lambda x: datetime.strptime(x,'%Y-%m-%d'),DateType())
covid_df=covid_df.withColumn('date_obs',func(covid_df['new_date']))

#add covid19 label 
add_cl =  udf (lambda x: having_covid(x),StringType())
covid_df=covid_df.withColumn('caseslabel',add_cl(covid_df['confirmed']))


#create data frame
covid_df=covid_df.select('pays','date_obs','confirmed','recovered','deaths','caseslabel')

#save it on hdfs
#df_covid.coalesce(1).write.csv('hdfs://192.168.122.206:8020/data/covid/covid19data08_04_2020.csv',header=True) 

                                   
     

In [1]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext


spark = SparkSession \
    .builder \
    .appName("Pysparkexample") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


Exception: Java gateway process exited before sending its port number

In [3]:
covid_df.filter(covid_df.pays=='France').show(10)

+------+----------+---------+---------+------+----------+
|  pays|  date_obs|confirmed|recovered|deaths|caseslabel|
+------+----------+---------+---------+------+----------+
|France|2020-01-22|        0|        0|     0|         0|
|France|2020-01-23|        0|        0|     0|         0|
|France|2020-01-24|        2|        0|     0|         1|
|France|2020-01-25|        3|        0|     0|         1|
|France|2020-01-26|        3|        0|     0|         1|
|France|2020-01-27|        3|        0|     0|         1|
|France|2020-01-28|        4|        0|     0|         1|
|France|2020-01-29|        5|        0|     0|         1|
|France|2020-01-30|        5|        0|     0|         1|
|France|2020-01-31|        5|        0|     0|         1|
+------+----------+---------+---------+------+----------+
only showing top 10 rows

